In [1]:
import torch
from IPython.display import Image, clear_output  # to display images\n
from IPython.core.magic import register_line_cell_magic

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
torch.cuda.empty_cache()

Setup complete. Using torch 1.13.0+cu117 _CudaDeviceProperties(name='Quadro RTX 5000', major=7, minor=5, total_memory=16117MB, multi_processor_count=48)


In [7]:
# !pip install cython; pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# pip install -v -e .

In [ ]:
# expect folder with images and xmls
%cd my_dataset
%mkdir VOCdevkit/VOC2007
%cp images/ VOC2007/JPEGImages/
%cp annotations/xmls/ VOC2007/Annotations/
!python voc_txt_usa.py "VOCdevkit/VOC2007"
!ln -s ./VOCdevkit/VOC2007 ~YOLOX/datasets/VOCdevkit/VOC2007

In [20]:
# %cd ~/YOLOX
# %rm -rf datasets/*

# !ln -s ~/YOLOX/Pothole-US/train/ ./datasets/VOCdevkit

In [ ]:
# %cd ~/YOLOX
# %mkdir ./datasets/VOCdevkit/VOC2007
# %mv datasets/VOCdevkit/images/ datasets/VOCdevkit/VOC2007/JPEGImages/
# %mv datasets/VOCdevkit/annotations/xmls/ datasets/VOCdevkit/VOC2007/Annotations/

# !python voc_txt_usa.py "datasets/VOCdevkit/VOC2007"

In [21]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w+') as f:
        f.write(cell.format(**globals()))

In [22]:
%%writetemplate yolox/data/datasets/voc_classes.py

VOC_CLASSES = (
    "D00",
    "D10",
    "D20",
    "D40",
)

In [23]:
%%writetemplate yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
    "D00",
    "D10",
    "D20",
    "D40",
)

In [6]:
NUM_CLASSES = 4
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "exps/example/yolox_voc/yolox_voc_s.py"

In [ ]:
!python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -b 32 --fp16 -o -c yolox_s.pth -l tensorboard

In [ ]:
MODEL_PATH = "./YOLOX_outputs/yolox_voc_s/best_ckpt.pth"
!python tools/eval.py -n  yolox-s -c {MODEL_PATH} -b 32 --conf 0.001 -f exps/example/yolox_voc/yolox_voc_s.py

In [ ]:
print("demo")
!python tools/demo.py image -f ./exps/example/yolox_voc/yolox_voc_s.py -c ./YOLOX_outputs/yolox_voc_s/best_ckpt.pth --path ./Pothole-7/test --conf 0.25 --nms 0.45 --tsize 640 --save_result --device cpu

# from PIL import Image
# OUTPUT_IMAGE_PATH = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/vis_res/2021_08_01_19_51_59/BloodImage_00057_jpg.rf.1ee93e9ec4d76cfaddaa7df70456c376.jpg" 
# Image.open(OUTPUT_IMAGE_PATH)